## a) Implementacja binarnej reprezentacji chromosomu + konfiguracja dokładności
- `bits` - liczba bitów chromosomu, liczba całkowita
- `quantity` - liczba chromosomów, liczba całkowita

In [7]:
import numpy as np

In [8]:
def gen_chromosome(bits, quantity = 1):
    return np.random.randint(0,2, size=(quantity, bits))

In [9]:
print(gen_chromosome(9))

[[1 0 1 1 0 1 1 1 0]]


## b) Implementacja konfiguracji wielkości populacji
- `bits` - liczba bitów chromosomu, liczba całkowita
- `popSize` - liczba chromosomów w populacji, liczba całkowita

In [10]:
def gen_population(bits, popSize):
    return gen_chromosome(bits, popSize)

In [14]:
population = gen_population(9, 20)
print(population)

[[1 0 0 0 0 1 1 1 0]
 [1 0 1 1 0 0 0 1 1]
 [0 0 0 1 1 1 1 1 1]
 [1 0 0 1 0 0 0 0 0]
 [1 0 0 0 0 0 1 1 0]
 [1 0 0 1 0 1 0 0 0]
 [1 0 1 1 1 0 0 0 0]
 [0 1 0 0 1 1 0 0 0]
 [1 1 0 1 1 0 1 1 1]
 [1 0 1 1 1 0 1 0 0]
 [1 1 0 1 1 1 0 1 0]
 [0 0 0 0 1 1 0 1 1]
 [0 0 0 1 0 1 0 1 1]
 [0 0 1 0 0 1 1 0 1]
 [0 0 1 1 0 0 1 0 1]
 [0 0 0 0 0 0 1 1 1]
 [1 0 1 1 0 0 0 1 0]
 [1 0 1 0 1 1 0 1 0]
 [1 0 1 0 1 1 1 1 0]
 [1 1 1 0 0 1 1 0 0]]


In [ ]:
## b) Implementacja rozkodowania osobników - konwersji osobnika z reprezentacji binarnej do reprezentacji dziesiętnej
- `binary` - osobnik w reprezentacji binarnej, liczba całkowita
- `decimal` - liczba chromosomów w populacji, liczba całkowita


In [52]:
def toDecimal(binary):
    decimal = 0
    position = len(binary)
    for bit in binary:
        position -= 1
        decimal += bit * pow(2, position)
    return decimal

In [53]:
toDecimal(population[0])

270

## c) Implementacja konfiguracji liczby epok